# Modelagem

* O objetivo da modelagem é prever o preço do carros com as colunas, por isso se trata de um problema de regressão poís a váriável é contínua

* Primeiro passo é dividir entre treino e teste, o teste não será olhado em todo o processo, pois é para simular como seria na vida real, onde não se tem a váriavel resposta. Isso diminui a chance de ocorrer Overfitting ou underfitting

* Na preparação dos dados será iniciado com as váriaveis com dados faltantes, o número de fotos será preenchido com 0 onde não tem dados, se não foi preenchido provável que nem tenha foto. Os restantes das váriaveis serão transformadas em váriaveis Dummy, onde se ocorreu é 1 e se não é 0, foi observado que essas váriaveis só tem preenchida se aconteceu, se não aconteceu é dado faltante, se é dado faltante não ocorreu o evento e é zero

* Nas variável binarias, Blindado, entrega_delivery e troca, será ajustado como: Se o evento ocorereu é 1, se não é 0. Na váriável tipo de vendedor, PJ sérá 1 e PF será 0, ficará uma variável dummy

* Nas variavéis categóricas onde se tem mais de dois valores possíveis será utilizado o Target Encoder, só de carro tem 457, ser for utilizado o one hote coder terá 457 colunas fora as outras variáveis, fica com muitas colunas, utilizado o Target Encoder não irá aumentar o número de colunas.


* O metódo de Target Encoder agrupa a categoria e substitui pela média da variável alvo, no caso preço dos carros

* Será feito os ajustes necessários antes de iniciar a modelagem

* Será utilizado o LGBMRegressor, modelo do tipo boosting

* As métricas de avaliação será o MSE e MAE, Erro Quadrático Médio(MSE) é uma métrica que penaliza erros por elevar ao quadrado. Erro Absoluto Médio(MAE) é uma métrica que é melhor para interpretar pois não é elevado ao quadrado

* Será utilizado o Recursive Feature Elimination para a Feature Selection. Este metódo consiste em treinar os modelos com todas as features, ordenar as features de acordo com a importância, rodar de volta sem a feature menos importante, calcular o desempenho, se a diferença estiver abaixo de um valor a feature é retirada. Esse processo se repete algumas vezes

* Após utilizar o Recursive Feature Elimination, se for necessário será retirado features. Após isso será tunado os hiperparâmetros do LGBM

* Após esses procesos a base de teste será preparada e será feita a previsão utilizando o modelo escolhido.


# Preparação inicial

In [400]:
import pandas as pd
import numpy as np


In [401]:
df = pd.read_csv("/content/cars_train.csv", sep=";",encoding = 'ISO-8859-1')
df.head()

,id,num_fotos,marca,modelo,versao,ano_de_fabricacao,ano_modelo,hodometro,cambio,num_portas,...,elegivel_revisao,dono_aceita_troca,veiculo_único_dono,revisoes_concessionaria,ipva_pago,veiculo_licenciado,garantia_de_fábrica,revisoes_dentro_agenda,veiculo_alienado,preco
0,"3,01E+38",8.0,NISSAN,KICKS,1.6 16V FLEXSTART SL 4P XTRONIC,2017,2017.0,67772.0,CVT,4,...,False,NaN,NaN,Todas as revisões feitas pela concessionária,IPVA pago,Licenciado,NaN,NaN,NaN,"74732,59"
1,"2,80E+38",8.0,JEEP,COMPASS,2.0 16V FLEX LIMITED AUTOMÁTICO,2017,2017.0,62979.0,Automática,4,...,False,Aceita troca,NaN,NaN,IPVA pago,NaN,NaN,NaN,NaN,"81965,33"
2,"5,64E+37",16.0,KIA,SORENTO,2.4 16V GASOLINA EX 7L AWD AUTOMÁTICO,2018,2019.0,44070.0,Automática,4,...,False,Aceita troca,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"162824,81"
3,"5,69E+37",14.0,VOLKSWAGEN,AMAROK,2.0 HIGHLINE 4X4 CD 16V TURBO INTERCOOLER DIES...,2013,2015.0,85357.0,Automática,4,...,False,Aceita troca,NaN,NaN,IPVA pago,Licenciado,NaN,NaN,NaN,"123681,36"
4,"3,39E+38",8.0,SSANGYONG,KORANDO,2.0 GLS 4X4 16V TURBO DIESEL 4P AUTOMÁTICO,2013,2015.0,71491.0,Automática,4,...,False,NaN,NaN,Todas as revisões feitas pela concessionária,NaN,NaN,Garantia de fábrica,Todas as revisões feitas pela agenda do carro,NaN,"82419,76"


In [402]:
df['preco'] = df['preco'].str.replace(',','.')
df["preco"] = df['preco'].astype(float)

In [403]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29584 entries, 0 to 29583
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       29584 non-null  object 
 1   num_fotos                29407 non-null  float64
 2   marca                    29584 non-null  object 
 3   modelo                   29584 non-null  object 
 4   versao                   29584 non-null  object 
 5   ano_de_fabricacao        29584 non-null  int64  
 6   ano_modelo               29584 non-null  float64
 7   hodometro                29584 non-null  float64
 8   cambio                   29584 non-null  object 
 9   num_portas               29584 non-null  int64  
 10  tipo                     29584 non-null  object 
 11  blindado                 29584 non-null  object 
 12  cor                      29584 non-null  object 
 13  tipo_vendedor            29584 non-null  object 
 14  cidade_vendedor       

In [404]:
df.shape

(29584, 29)

In [405]:
X = df.drop(['preco',"id","tipo","marca",'veiculo_alienado', 'elegivel_revisao'], axis = 1)
y = df['preco']


In [406]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.30,random_state=30)

In [407]:
X_train.head()

,num_fotos,modelo,versao,ano_de_fabricacao,ano_modelo,hodometro,cambio,num_portas,blindado,cor,...,anunciante,entrega_delivery,troca,dono_aceita_troca,veiculo_único_dono,revisoes_concessionaria,ipva_pago,veiculo_licenciado,garantia_de_fábrica,revisoes_dentro_agenda
23344,8.0,RENEGADE,1.8 16V FLEX SPORT 4P AUTOMÁTICO,2018,2018.0,41728.0,Automática,4,N,Branco,...,Pessoa Física,False,False,NaN,NaN,NaN,IPVA pago,Licenciado,NaN,NaN
7877,8.0,JETTA,1.4 250 TSI TOTAL FLEX TIPTRONIC,2018,2018.0,78971.0,Automática,4,N,Branco,...,Pessoa Física,False,False,NaN,Único dono,NaN,IPVA pago,Licenciado,NaN,NaN
18110,8.0,COMPASS,2.0 16V FLEX LIMITED AUTOMÁTICO,2019,2019.0,25537.0,Automática,4,N,Branco,...,Pessoa Física,False,False,NaN,Único dono,Todas as revisões feitas pela concessionária,IPVA pago,Licenciado,Garantia de fábrica,Todas as revisões feitas pela agenda do carro
3174,9.0,GOL,1.6 16V MSI TOTALFLEX 4P AUTOMÁTICO,2021,2021.0,59527.0,Automática,4,N,Prata,...,Loja,False,False,Aceita troca,Único dono,NaN,IPVA pago,Licenciado,NaN,NaN
22389,8.0,HB20X,1.6 16V STYLE FLEX 4P AUTOMÁTICO,2017,2018.0,53298.0,Automática,4,N,Branco,...,Pessoa Física,False,False,NaN,Único dono,Todas as revisões feitas pela concessionária,IPVA pago,Licenciado,NaN,Todas as revisões feitas pela agenda do carro


# preenchmineto dos valores faltantes

In [408]:
X.isnull().sum()

num_fotos                    177
modelo                         0
versao                         0
ano_de_fabricacao              0
ano_modelo                     0
hodometro                      0
cambio                         0
num_portas                     0
blindado                       0
cor                            0
tipo_vendedor                  0
cidade_vendedor                0
estado_vendedor                0
anunciante                     0
entrega_delivery               0
troca                          0
dono_aceita_troca           7662
veiculo_único_dono         19161
revisoes_concessionaria    20412
ipva_pago                   9925
veiculo_licenciado         13678
garantia_de_fábrica        25219
revisoes_dentro_agenda     23674
dtype: int64

## num-fotos

In [409]:
X_train["num_fotos"].fillna(0, inplace=True)
X_test["num_fotos"].fillna(0, inplace=True)

In [410]:
X_train["num_fotos"].isnull().sum()

0

## dono_aceita_troca

In [411]:
X_train["dono_aceita_troca"].fillna(0, inplace=True)
X_test["dono_aceita_troca"].fillna(0, inplace=True)

In [412]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29584 entries, 0 to 29583
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       29584 non-null  object 
 1   num_fotos                29407 non-null  float64
 2   marca                    29584 non-null  object 
 3   modelo                   29584 non-null  object 
 4   versao                   29584 non-null  object 
 5   ano_de_fabricacao        29584 non-null  int64  
 6   ano_modelo               29584 non-null  float64
 7   hodometro                29584 non-null  float64
 8   cambio                   29584 non-null  object 
 9   num_portas               29584 non-null  int64  
 10  tipo                     29584 non-null  object 
 11  blindado                 29584 non-null  object 
 12  cor                      29584 non-null  object 
 13  tipo_vendedor            29584 non-null  object 
 14  cidade_vendedor       

In [413]:
X_train["dono_aceita_troca"]

23344               0
7877                0
18110               0
3174     Aceita troca
22389               0
             ...     
500      Aceita troca
12077    Aceita troca
15277               0
4517                0
5925     Aceita troca
Name: dono_aceita_troca, Length: 20708, dtype: object

In [414]:
X_train["dono_aceita_troca"][X_train["dono_aceita_troca"] == 'Aceita troca'] = 1
X_test["dono_aceita_troca"][X_test["dono_aceita_troca"] == 'Aceita troca'] = 1

<ipython-input-414-784e045d0e30>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train["dono_aceita_troca"][X_train["dono_aceita_troca"] == 'Aceita troca'] = 1
<ipython-input-414-784e045d0e30>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test["dono_aceita_troca"][X_test["dono_aceita_troca"] == 'Aceita troca'] = 1


In [415]:
X_train["dono_aceita_troca"]

23344    0
7877     0
18110    0
3174     1
22389    0
        ..
500      1
12077    1
15277    0
4517     0
5925     1
Name: dono_aceita_troca, Length: 20708, dtype: object

## veiculo_único_dono

In [416]:
X_train["veiculo_único_dono"]

23344           NaN
7877     Único dono
18110    Único dono
3174     Único dono
22389    Único dono
            ...    
500             NaN
12077           NaN
15277    Único dono
4517            NaN
5925            NaN
Name: veiculo_único_dono, Length: 20708, dtype: object

In [417]:
X_train["veiculo_único_dono"].fillna(0, inplace=True)
X_test["veiculo_único_dono"].fillna(0, inplace=True)

In [418]:
X_train["veiculo_único_dono"][X_train["veiculo_único_dono"] == 'Único dono'] = 1
X_test["veiculo_único_dono"][X_test["veiculo_único_dono"] == 'Único dono'] = 1

<ipython-input-418-d711a6df042f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train["veiculo_único_dono"][X_train["veiculo_único_dono"] == 'Único dono'] = 1
<ipython-input-418-d711a6df042f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test["veiculo_único_dono"][X_test["veiculo_único_dono"] == 'Único dono'] = 1


In [419]:
X_train["veiculo_único_dono"]

23344    0
7877     1
18110    1
3174     1
22389    1
        ..
500      0
12077    0
15277    1
4517     0
5925     0
Name: veiculo_único_dono, Length: 20708, dtype: object

## revisoes_concessionaria

In [420]:
X_train["revisoes_concessionaria"]

23344                                             NaN
7877                                              NaN
18110    Todas as revisões feitas pela concessionária
3174                                              NaN
22389    Todas as revisões feitas pela concessionária
                             ...                     
500                                               NaN
12077                                             NaN
15277                                             NaN
4517     Todas as revisões feitas pela concessionária
5925                                              NaN
Name: revisoes_concessionaria, Length: 20708, dtype: object

In [421]:
X_train["revisoes_concessionaria"].fillna(0, inplace=True)
X_test["revisoes_concessionaria"].fillna(0, inplace=True)

In [422]:
X_train["revisoes_concessionaria"][X_train["revisoes_concessionaria"] == 'Todas as revisões feitas pela concessionária'] = 1
X_test["revisoes_concessionaria"][X_test["revisoes_concessionaria"] == 'Todas as revisões feitas pela concessionária'] = 1

<ipython-input-422-b9babc2580b7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train["revisoes_concessionaria"][X_train["revisoes_concessionaria"] == 'Todas as revisões feitas pela concessionária'] = 1
<ipython-input-422-b9babc2580b7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test["revisoes_concessionaria"][X_test["revisoes_concessionaria"] == 'Todas as revisões feitas pela concessionária'] = 1


In [423]:
X_train["revisoes_concessionaria"]

23344    0
7877     0
18110    1
3174     0
22389    1
        ..
500      0
12077    0
15277    0
4517     1
5925     0
Name: revisoes_concessionaria, Length: 20708, dtype: object

## ipva_pago

In [424]:
X_train["ipva_pago"]

23344    IPVA pago
7877     IPVA pago
18110    IPVA pago
3174     IPVA pago
22389    IPVA pago
           ...    
500            NaN
12077          NaN
15277    IPVA pago
4517     IPVA pago
5925           NaN
Name: ipva_pago, Length: 20708, dtype: object

In [425]:
X_train["ipva_pago"].fillna(0, inplace=True)
X_test["ipva_pago"].fillna(0, inplace=True)

In [426]:
X_train["ipva_pago"][X_train["ipva_pago"] == 'IPVA pago'] = 1
X_test["ipva_pago"][X_test["ipva_pago"] == 'IPVA pago'] = 1

<ipython-input-426-ded338bfaa06>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train["ipva_pago"][X_train["ipva_pago"] == 'IPVA pago'] = 1
<ipython-input-426-ded338bfaa06>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test["ipva_pago"][X_test["ipva_pago"] == 'IPVA pago'] = 1


In [427]:
X_train["ipva_pago"]

23344    1
7877     1
18110    1
3174     1
22389    1
        ..
500      0
12077    0
15277    1
4517     1
5925     0
Name: ipva_pago, Length: 20708, dtype: object

## veiculo_licenciado

In [428]:
X_train["veiculo_licenciado"]

23344    Licenciado
7877     Licenciado
18110    Licenciado
3174     Licenciado
22389    Licenciado
            ...    
500             NaN
12077           NaN
15277    Licenciado
4517     Licenciado
5925            NaN
Name: veiculo_licenciado, Length: 20708, dtype: object

In [429]:
X_train["veiculo_licenciado"].fillna(0, inplace=True)
X_test["veiculo_licenciado"].fillna(0, inplace=True)

In [430]:
X_train["veiculo_licenciado"][X_train["veiculo_licenciado"] == 'Licenciado'] = 1
X_test["veiculo_licenciado"][X_test["veiculo_licenciado"] == 'Licenciado'] = 1

<ipython-input-430-9165953e6dba>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train["veiculo_licenciado"][X_train["veiculo_licenciado"] == 'Licenciado'] = 1
<ipython-input-430-9165953e6dba>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test["veiculo_licenciado"][X_test["veiculo_licenciado"] == 'Licenciado'] = 1


In [431]:
X_train["veiculo_licenciado"]

23344    1
7877     1
18110    1
3174     1
22389    1
        ..
500      0
12077    0
15277    1
4517     1
5925     0
Name: veiculo_licenciado, Length: 20708, dtype: object

## garantia_de_fábrica

In [432]:
X_train["garantia_de_fábrica"]

23344                    NaN
7877                     NaN
18110    Garantia de fábrica
3174                     NaN
22389                    NaN
                ...         
500                      NaN
12077                    NaN
15277                    NaN
4517                     NaN
5925                     NaN
Name: garantia_de_fábrica, Length: 20708, dtype: object

In [433]:
X_train["garantia_de_fábrica"].fillna(0, inplace=True)
X_test["garantia_de_fábrica"].fillna(0, inplace=True)

In [434]:
X_train["garantia_de_fábrica"][X_train["garantia_de_fábrica"] == 'Garantia de fábrica'] = 1
X_test["garantia_de_fábrica"][X_test["garantia_de_fábrica"] == 'Garantia de fábrica'] = 1

<ipython-input-434-7d06b151d6c1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train["garantia_de_fábrica"][X_train["garantia_de_fábrica"] == 'Garantia de fábrica'] = 1
<ipython-input-434-7d06b151d6c1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test["garantia_de_fábrica"][X_test["garantia_de_fábrica"] == 'Garantia de fábrica'] = 1


In [435]:
X_train["garantia_de_fábrica"]

23344    0
7877     0
18110    1
3174     0
22389    0
        ..
500      0
12077    0
15277    0
4517     0
5925     0
Name: garantia_de_fábrica, Length: 20708, dtype: object

## revisoes_dentro_agenda

In [436]:
X_train["revisoes_dentro_agenda"]

23344                                              NaN
7877                                               NaN
18110    Todas as revisões feitas pela agenda do carro
3174                                               NaN
22389    Todas as revisões feitas pela agenda do carro
                             ...                      
500                                                NaN
12077                                              NaN
15277                                              NaN
4517                                               NaN
5925                                               NaN
Name: revisoes_dentro_agenda, Length: 20708, dtype: object

In [437]:
X_train["revisoes_dentro_agenda"].fillna(0, inplace=True)
X_test["revisoes_dentro_agenda"].fillna(0, inplace=True)

In [438]:
X_train["revisoes_dentro_agenda"][X_train["revisoes_dentro_agenda"] == 'Todas as revisões feitas pela agenda do carro'] = 1
X_test["revisoes_dentro_agenda"][X_test["revisoes_dentro_agenda"] == 'Todas as revisões feitas pela agenda do carro'] = 1

<ipython-input-438-81bd774d76d1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train["revisoes_dentro_agenda"][X_train["revisoes_dentro_agenda"] == 'Todas as revisões feitas pela agenda do carro'] = 1
<ipython-input-438-81bd774d76d1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test["revisoes_dentro_agenda"][X_test["revisoes_dentro_agenda"] == 'Todas as revisões feitas pela agenda do carro'] = 1


In [439]:
X_train["revisoes_dentro_agenda"]

23344    0
7877     0
18110    1
3174     0
22389    1
        ..
500      0
12077    0
15277    0
4517     0
5925     0
Name: revisoes_dentro_agenda, Length: 20708, dtype: object

# Ajustando variáveis Binárias

## Blindado

In [441]:
X_train

,num_fotos,modelo,versao,ano_de_fabricacao,ano_modelo,hodometro,cambio,num_portas,blindado,cor,...,anunciante,entrega_delivery,troca,dono_aceita_troca,veiculo_único_dono,revisoes_concessionaria,ipva_pago,veiculo_licenciado,garantia_de_fábrica,revisoes_dentro_agenda
23344,8.0,RENEGADE,1.8 16V FLEX SPORT 4P AUTOMÁTICO,2018,2018.0,41728.0,Automática,4,N,Branco,...,Pessoa Física,False,False,0,0,0,1,1,0,0
7877,8.0,JETTA,1.4 250 TSI TOTAL FLEX TIPTRONIC,2018,2018.0,78971.0,Automática,4,N,Branco,...,Pessoa Física,False,False,0,1,0,1,1,0,0
18110,8.0,COMPASS,2.0 16V FLEX LIMITED AUTOMÁTICO,2019,2019.0,25537.0,Automática,4,N,Branco,...,Pessoa Física,False,False,0,1,1,1,1,1,1
3174,9.0,GOL,1.6 16V MSI TOTALFLEX 4P AUTOMÁTICO,2021,2021.0,59527.0,Automática,4,N,Prata,...,Loja,False,False,1,1,0,1,1,0,0
22389,8.0,HB20X,1.6 16V STYLE FLEX 4P AUTOMÁTICO,2017,2018.0,53298.0,Automática,4,N,Branco,...,Pessoa Física,False,False,0,1,1,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,8.0,RENEGADE,1.8 16V FLEX 4P AUTOMÁTICO,2019,2019.0,31343.0,Automática,4,N,Branco,...,Pessoa Física,False,False,1,0,0,0,0,0,0
12077,16.0,RENEGADE,2.0 16V TURBO DIESEL SPORT 4P 4X4 AUTOMÁTICO,2014,2014.0,123251.0,Automática,4,N,Branco,...,Loja,True,True,1,0,0,0,0,0,0
15277,8.0,DUSTER,2.0 DYNAMIQUE 4X2 16V FLEX 4P AUTOMÁTICO,2016,2017.0,69129.0,Automática,4,N,Branco,...,Pessoa Física,False,False,0,1,0,1,1,0,0
4517,8.0,HB20,1.6 COMFORT PLUS 16V FLEX 4P MANUAL,2015,2015.0,73560.0,Manual,4,N,Branco,...,Pessoa Física,False,False,0,0,1,1,1,0,0


In [442]:
X_train['blindado']

23344    N
7877     N
18110    N
3174     N
22389    N
        ..
500      N
12077    N
15277    N
4517     N
5925     N
Name: blindado, Length: 20708, dtype: object

In [443]:
X_train['blindado'].value_counts()

N    20540
S      168
Name: blindado, dtype: int64

In [444]:
X_train["blindado"][X_train["blindado"] == 'N'] = 0
X_test["blindado"][X_test["blindado"] == 'N'] = 0

X_train["blindado"][X_train["blindado"] == 'S'] = 1
X_test["blindado"][X_test["blindado"] == 'S'] = 1

<ipython-input-444-50edddd37f92>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train["blindado"][X_train["blindado"] == 'N'] = 0
<ipython-input-444-50edddd37f92>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test["blindado"][X_test["blindado"] == 'N'] = 0
<ipython-input-444-50edddd37f92>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train["blindado"][X_train["blindado"] == 'S'] = 1
<ipython-input-444-50edddd37f92>:5: Set

In [445]:
X_train['blindado'].value_counts()

0    20540
1      168
Name: blindado, dtype: int64

## entrega_delivery

In [446]:
X_train['entrega_delivery']

23344    False
7877     False
18110    False
3174     False
22389    False
         ...  
500      False
12077     True
15277    False
4517     False
5925     False
Name: entrega_delivery, Length: 20708, dtype: bool

In [447]:
X_train['entrega_delivery'].value_counts()

False    16469
True      4239
Name: entrega_delivery, dtype: int64

In [448]:
X_train["entrega_delivery"][X_train["entrega_delivery"] == False] = 0
X_test["entrega_delivery"][X_test["entrega_delivery"] == False] = 0

X_train["entrega_delivery"][X_train["entrega_delivery"] == True] = 1
X_test["entrega_delivery"][X_test["entrega_delivery"] == True] = 1

<ipython-input-448-cfcca4aab687>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train["entrega_delivery"][X_train["entrega_delivery"] == False] = 0
<ipython-input-448-cfcca4aab687>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test["entrega_delivery"][X_test["entrega_delivery"] == False] = 0


In [449]:
X_train['entrega_delivery'].value_counts()

0    16469
1     4239
Name: entrega_delivery, dtype: int64

In [450]:
X_train['entrega_delivery']

23344    0
7877     0
18110    0
3174     0
22389    0
        ..
500      0
12077    1
15277    0
4517     0
5925     0
Name: entrega_delivery, Length: 20708, dtype: object

## Troca

In [451]:
X_train['troca']

23344    False
7877     False
18110    False
3174     False
22389    False
         ...  
500      False
12077     True
15277    False
4517     False
5925     False
Name: troca, Length: 20708, dtype: bool

In [452]:
X_train['troca'].value_counts()

False    17128
True      3580
Name: troca, dtype: int64

In [453]:
X_train["troca"][X_train["troca"] == False] = 0
X_test["troca"][X_test["troca"] == False] = 0

X_train["troca"][X_train["troca"] == True] = 1
X_test["troca"][X_test["troca"] == True] = 1

<ipython-input-453-4207a81f30c1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train["troca"][X_train["troca"] == False] = 0
<ipython-input-453-4207a81f30c1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test["troca"][X_test["troca"] == False] = 0


In [454]:
X_train['troca'].value_counts()

0    17128
1     3580
Name: troca, dtype: int64

In [455]:
X_train.head()

,num_fotos,modelo,versao,ano_de_fabricacao,ano_modelo,hodometro,cambio,num_portas,blindado,cor,...,anunciante,entrega_delivery,troca,dono_aceita_troca,veiculo_único_dono,revisoes_concessionaria,ipva_pago,veiculo_licenciado,garantia_de_fábrica,revisoes_dentro_agenda
23344,8.0,RENEGADE,1.8 16V FLEX SPORT 4P AUTOMÁTICO,2018,2018.0,41728.0,Automática,4,0,Branco,...,Pessoa Física,0,0,0,0,0,1,1,0,0
7877,8.0,JETTA,1.4 250 TSI TOTAL FLEX TIPTRONIC,2018,2018.0,78971.0,Automática,4,0,Branco,...,Pessoa Física,0,0,0,1,0,1,1,0,0
18110,8.0,COMPASS,2.0 16V FLEX LIMITED AUTOMÁTICO,2019,2019.0,25537.0,Automática,4,0,Branco,...,Pessoa Física,0,0,0,1,1,1,1,1,1
3174,9.0,GOL,1.6 16V MSI TOTALFLEX 4P AUTOMÁTICO,2021,2021.0,59527.0,Automática,4,0,Prata,...,Loja,0,0,1,1,0,1,1,0,0
22389,8.0,HB20X,1.6 16V STYLE FLEX 4P AUTOMÁTICO,2017,2018.0,53298.0,Automática,4,0,Branco,...,Pessoa Física,0,0,0,1,1,1,1,0,1


## tipo vendedor

In [456]:
X_train["tipo_vendedor"].value_counts()

PF    12484
PJ     8224
Name: tipo_vendedor, dtype: int64

In [457]:
X_train["tipo_vendedor"][X_train["tipo_vendedor"] == 'PF'] = 0
X_test["tipo_vendedor"][X_test["tipo_vendedor"] == 'PF'] = 0

X_train["tipo_vendedor"][X_train["tipo_vendedor"] == 'PJ'] = 1
X_test["tipo_vendedor"][X_test["tipo_vendedor"] == 'PJ'] = 1

<ipython-input-457-80e37aa85d08>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train["tipo_vendedor"][X_train["tipo_vendedor"] == 'PF'] = 0
<ipython-input-457-80e37aa85d08>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test["tipo_vendedor"][X_test["tipo_vendedor"] == 'PF'] = 0
<ipython-input-457-80e37aa85d08>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train["tipo_vendedor"][X_train["tipo_vendedor"] == 'PJ'] = 1
<ipyth

In [458]:
X_train["tipo_vendedor"].value_counts()

0    12484
1     8224
Name: tipo_vendedor, dtype: int64

## Ajustando tipo de coluna

In [459]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20708 entries, 23344 to 5925
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   num_fotos                20708 non-null  float64
 1   modelo                   20708 non-null  object 
 2   versao                   20708 non-null  object 
 3   ano_de_fabricacao        20708 non-null  int64  
 4   ano_modelo               20708 non-null  float64
 5   hodometro                20708 non-null  float64
 6   cambio                   20708 non-null  object 
 7   num_portas               20708 non-null  int64  
 8   blindado                 20708 non-null  object 
 9   cor                      20708 non-null  object 
 10  tipo_vendedor            20708 non-null  object 
 11  cidade_vendedor          20708 non-null  object 
 12  estado_vendedor          20708 non-null  object 
 13  anunciante               20708 non-null  object 
 14  entrega_delivery   

In [481]:
X_train[["troca","dono_aceita_troca","veiculo_único_dono",
"revisoes_concessionaria","ipva_pago","veiculo_licenciado",
"garantia_de_fábrica","revisoes_dentro_agenda","entrega_delivery","tipo_vendedor","blindado"]]= X_train[["troca","dono_aceita_troca","veiculo_único_dono",
"revisoes_concessionaria","ipva_pago","veiculo_licenciado",
"garantia_de_fábrica","revisoes_dentro_agenda","entrega_delivery","tipo_vendedor","blindado"]].astype(float)

X_test[["troca","dono_aceita_troca","veiculo_único_dono",
"revisoes_concessionaria","ipva_pago","veiculo_licenciado",
"garantia_de_fábrica","revisoes_dentro_agenda","entrega_delivery","tipo_vendedor","blindado"]]= X_test[["troca","dono_aceita_troca","veiculo_único_dono",
"revisoes_concessionaria","ipva_pago","veiculo_licenciado",
"garantia_de_fábrica","revisoes_dentro_agenda","entrega_delivery","tipo_vendedor","blindado"]].astype(float)

In [461]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20708 entries, 23344 to 5925
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   num_fotos                20708 non-null  float64
 1   modelo                   20708 non-null  object 
 2   versao                   20708 non-null  object 
 3   ano_de_fabricacao        20708 non-null  int64  
 4   ano_modelo               20708 non-null  float64
 5   hodometro                20708 non-null  float64
 6   cambio                   20708 non-null  object 
 7   num_portas               20708 non-null  int64  
 8   blindado                 20708 non-null  object 
 9   cor                      20708 non-null  object 
 10  tipo_vendedor            20708 non-null  float64
 11  cidade_vendedor          20708 non-null  object 
 12  estado_vendedor          20708 non-null  object 
 13  anunciante               20708 non-null  object 
 14  entrega_delivery   

## Variavéis categóricas

In [130]:
pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 3.1 MB/s eta 0:00:00


In [375]:
from category_encoders import TargetEncoder

In [464]:
TE_encoder = TargetEncoder()
train_te = TE_encoder.fit_transform(X_train[["modelo","versao","cor","cidade_vendedor","estado_vendedor","anunciante",'cambio']], y_train)
test_te = TE_encoder.transform(X_test[["modelo","versao","cor","cidade_vendedor","estado_vendedor","anunciante","cambio"]])

train_te.head()

,modelo,versao,cor,cidade_vendedor,estado_vendedor,anunciante,cambio
23344,100374.114742,106044.724243,130277.172282,126311.487337,129709.574763,138703.595663,145945.09097
7877,117640.470418,126124.354834,130277.172282,126747.551476,140517.085527,138703.595663,145945.09097
18110,148854.518762,136553.331206,130277.172282,138233.886991,129709.574763,138703.595663,145945.09097
3174,88293.783591,109803.703889,96184.798740,135268.363601,144455.945767,123044.935426,145945.09097
22389,114208.426709,122290.352296,130277.172282,164681.673988,153076.079648,138703.595663,145945.09097


In [465]:
X_train = X_train.drop(["modelo","versao","cor","cidade_vendedor","estado_vendedor","anunciante",'cambio'], axis = 1)
X_test = X_test.drop(["modelo","versao","cor","cidade_vendedor","estado_vendedor","anunciante",'cambio'], axis = 1)

In [467]:
X_train.shape

(20708, 16)

In [468]:
train_te.shape

(20708, 7)

In [515]:
X_train=X_train.merge(train_te, left_index=True,right_index=True)

In [517]:
X_train

,num_fotos,ano_de_fabricacao,ano_modelo,hodometro,num_portas,blindado,tipo_vendedor,entrega_delivery,troca,dono_aceita_troca,...,veiculo_licenciado,garantia_de_fábrica,revisoes_dentro_agenda,modelo,versao,cor,cidade_vendedor,estado_vendedor,anunciante,cambio
23344,8.0,2018,2018.0,41728.0,4,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,100374.114742,106044.724243,130277.172282,126311.487337,129709.574763,138703.595663,145945.090970
7877,8.0,2018,2018.0,78971.0,4,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,117640.470418,126124.354834,130277.172282,126747.551476,140517.085527,138703.595663,145945.090970
18110,8.0,2019,2019.0,25537.0,4,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,148854.518762,136553.331206,130277.172282,138233.886991,129709.574763,138703.595663,145945.090970
3174,9.0,2021,2021.0,59527.0,4,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,88293.783591,109803.703889,96184.798740,135268.363601,144455.945767,123044.935426,145945.090970
22389,8.0,2017,2018.0,53298.0,4,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,114208.426709,122290.352296,130277.172282,164681.673988,153076.079648,138703.595663,145945.090970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,8.0,2019,2019.0,31343.0,4,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,100374.114742,79187.544478,130277.172282,126311.487337,129709.574763,138703.595663,145945.090970
12077,16.0,2014,2014.0,123251.0,4,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,100374.114742,121777.182459,130277.172282,132822.918819,135333.510055,123044.935426,145945.090970
15277,8.0,2016,2017.0,69129.0,4,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,76917.526527,116111.028907,130277.172282,132822.918819,135333.510055,138703.595663,145945.090970
4517,8.0,2015,2015.0,73560.0,4,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,66160.193250,64599.011515,130277.172282,126311.487337,129709.574763,138703.595663,81204.254603


In [518]:
X_test=X_test.merge(test_te, left_index=True,right_index=True)

In [519]:
X_test

,num_fotos,ano_de_fabricacao,ano_modelo,hodometro,num_portas,blindado,tipo_vendedor,entrega_delivery,troca,dono_aceita_troca,...,veiculo_licenciado,garantia_de_fábrica,revisoes_dentro_agenda,modelo,versao,cor,cidade_vendedor,estado_vendedor,anunciante,cambio
19849,8.0,2019,2019.0,26494.0,4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,75499.356815,72076.386254,130277.172282,126311.487337,129709.574763,138703.595663,145945.09097
22422,8.0,2019,2020.0,26169.0,4,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,248023.822657,261788.049728,130277.172282,189203.996144,164695.784652,138703.595663,145945.09097
18119,8.0,2013,2014.0,96112.0,4,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,158467.324809,138937.277146,136875.657908,147482.029776,144455.945767,138703.595663,145945.09097
28419,17.0,2016,2016.0,80136.0,4,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,150267.752490,132492.259714,130277.172282,132822.918819,129709.574763,123044.935426,145945.09097
13559,8.0,2021,2021.0,48194.0,4,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,1.0,124668.917873,127542.220631,196878.577634,162352.353451,129709.574763,138703.595663,145945.09097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9124,8.0,2021,2021.0,16610.0,4,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,124668.917873,127542.220631,196878.577634,132822.918819,129709.574763,138703.595663,145945.09097
19795,8.0,2019,2020.0,20313.0,4,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,124668.917873,127542.220631,130277.172282,132822.918819,135333.510055,138703.595663,145945.09097
17501,15.0,2020,2021.0,10716.0,4,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,84654.362525,93278.311179,130277.172282,126311.487337,129709.574763,132334.304452,145945.09097
17939,16.0,2018,2018.0,60968.0,4,0.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,75738.305769,101477.848409,130277.172282,132822.918819,135333.510055,123044.935426,145945.09097


# Modelo

## Modelo base

In [478]:
from lightgbm import LGBMRegressor

In [520]:
modelo=LGBMRegressor()
modelo.fit(X_train,y_train)

LGBMRegressor()

In [521]:
from sklearn.metrics import mean_squared_error

pred=modelo.predict(X_test)
MSE=mean_squared_error(y_test, pred)

print(f' O MSE é {MSE} ')

 O MSE é 1840742350.6496954 


## Feature selection

In [492]:
from feature_engine.selection import RecursiveFeatureElimination

In [536]:
tr = RecursiveFeatureElimination(estimator=modelo, scoring="neg_mean_squared_error", cv=10)

In [537]:
X_train_tr = tr.fit_transform(X_train, y_train)
X_test_tr = tr.transform(X_test)

In [538]:
tr.performance_drifts_

{'tipo_vendedor': -2019644.3994886875,
 'entrega_delivery': -6360320.098179817,
 'anunciante': 1520097.233633995,
 'troca': 1911907.6334285736,
 'garantia_de_fábrica': 2581555.0738904476,
 'cambio': 14251349.902880669,
 'ipva_pago': 455261.4022061825,
 'revisoes_dentro_agenda': 4687545.9644367695,
 'revisoes_concessionaria': -1016319.7247042656,
 'veiculo_único_dono': 10728340.24241519,
 'blindado': 14351053.036749363,
 'veiculo_licenciado': 13586048.816624403,
 'num_portas': 12561039.748705387,
 'dono_aceita_troca': 15023185.596983671,
 'estado_vendedor': 1608169.817100048,
 'num_fotos': 13286868.604623795,
 'cor': 27201702.60229826,
 'ano_de_fabricacao': 1589478.725447178,
 'ano_modelo': 25440653.06034279,
 'cidade_vendedor': 30139126.423529148,
 'hodometro': 138554609.08763838,
 'modelo': 132438778.58326268,
 'versao': 263349189.55323458}

In [540]:
tr.features_to_drop_

['tipo_vendedor', 'entrega_delivery', 'revisoes_concessionaria']

In [526]:
X_train_tr.head()

,num_fotos,ano_de_fabricacao,ano_modelo,hodometro,num_portas,blindado,tipo_vendedor,dono_aceita_troca,veiculo_único_dono,revisoes_concessionaria,veiculo_licenciado,garantia_de_fábrica,revisoes_dentro_agenda,modelo,versao,cor,cidade_vendedor,estado_vendedor,anunciante,cambio
23344,8.0,2018,2018.0,41728.0,4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,100374.114742,106044.724243,130277.172282,126311.487337,129709.574763,138703.595663,145945.09097
7877,8.0,2018,2018.0,78971.0,4,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,117640.470418,126124.354834,130277.172282,126747.551476,140517.085527,138703.595663,145945.09097
18110,8.0,2019,2019.0,25537.0,4,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,148854.518762,136553.331206,130277.172282,138233.886991,129709.574763,138703.595663,145945.09097
3174,9.0,2021,2021.0,59527.0,4,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,88293.783591,109803.703889,96184.798740,135268.363601,144455.945767,123044.935426,145945.09097
22389,8.0,2017,2018.0,53298.0,4,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,114208.426709,122290.352296,130277.172282,164681.673988,153076.079648,138703.595663,145945.09097


In [545]:
X_train_tr.shape

(20708, 20)

In [542]:
X_train = X_train.drop(['tipo_vendedor', 'entrega_delivery', 'revisoes_concessionaria'], axis = 1)
X_test = X_test.drop(['tipo_vendedor', 'entrega_delivery', 'revisoes_concessionaria'], axis = 1)

In [546]:
X_train.shape

(20708, 20)

In [543]:
X_train.head()

,num_fotos,ano_de_fabricacao,ano_modelo,hodometro,num_portas,blindado,troca,dono_aceita_troca,veiculo_único_dono,ipva_pago,veiculo_licenciado,garantia_de_fábrica,revisoes_dentro_agenda,modelo,versao,cor,cidade_vendedor,estado_vendedor,anunciante,cambio
23344,8.0,2018,2018.0,41728.0,4,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,100374.114742,106044.724243,130277.172282,126311.487337,129709.574763,138703.595663,145945.09097
7877,8.0,2018,2018.0,78971.0,4,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,117640.470418,126124.354834,130277.172282,126747.551476,140517.085527,138703.595663,145945.09097
18110,8.0,2019,2019.0,25537.0,4,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,148854.518762,136553.331206,130277.172282,138233.886991,129709.574763,138703.595663,145945.09097
3174,9.0,2021,2021.0,59527.0,4,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,88293.783591,109803.703889,96184.798740,135268.363601,144455.945767,123044.935426,145945.09097
22389,8.0,2017,2018.0,53298.0,4,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,114208.426709,122290.352296,130277.172282,164681.673988,153076.079648,138703.595663,145945.09097


In [550]:
from lightgbm import LGBMRegressor

## Tunagem de hiperparâmetro

In [ ]:
X_train_tr

In [556]:
params = {
    'max_depth': [3,5,7,9,11,15,20],
    'n_estimators':[50,100,150,200,300,400],
    'learning_rate': [0.05,0.1,0.2],
    'reg_alpha': [0,0.1,0.5,0.1,0.2],
    'reg_lambda': [0,0.1,0.5]
}
modelo=LGBMRegressor()
lgbm=RandomizedSearchCV(modelo, param_distributions = params,
                                   scoring = 'neg_mean_squared_error',
                                   cv = 100,
                                   verbose = 1,
                                   n_iter = 10,
                                   n_jobs = -1)
lgbm.fit(X_train,y_train)

Fitting 100 folds for each of 10 candidates, totalling 1000 fits


RandomizedSearchCV(cv=100, estimator=LGBMRegressor(), n_jobs=-1,
                   param_distributions={'learning_rate': [0.05, 0.1, 0.2],
                                        'max_depth': [3, 5, 7, 9, 11, 15, 20],
                                        'n_estimators': [50, 100, 150, 200, 300,
                                                         400],
                                        'reg_alpha': [0, 0.1, 0.5, 0.1, 0.2],
                                        'reg_lambda': [0, 0.1, 0.5]},
                   scoring='neg_mean_squared_error', verbose=1)

In [557]:
lgbm.best_params_

{'reg_lambda': 0.5,
 'reg_alpha': 0.2,
 'n_estimators': 400,
 'max_depth': 11,
 'learning_rate': 0.1}

In [560]:
from sklearn.metrics import mean_squared_error

pred_tu=lgbm.predict(X_test)
MSE_tu=mean_squared_error(y_test, pred_tu)

print(f' O MSE do modelo tunado é {MSE_tu} ')

 O MSE do modelo tunado é 1820988916.9360147 


In [561]:
print(f' O MSE do modelo base é {MSE} ')

 O MSE do modelo base é 1840742350.6496954 


In [562]:
from sklearn.metrics import mean_absolute_error

MAE = mean_absolute_error(y_test,pred)

MAE_tu = mean_absolute_error(y_test, pred_tu)


print(f' O MAE do modelo base é {MAE} ')
print(f' O MAE do modelo tunado é {MAE_tu} ')

 O MAE do modelo base é 27665.59413098994 
 O MAE do modelo tunado é 27382.85254039208 


# Previsão na base teste

In [567]:
df_test = pd.read_csv("/content/cars_test.csv", sep=";")
df_test.head()

,id,num_fotos,marca,modelo,versao,ano_de_fabricacao,ano_modelo,hodometro,cambio,num_portas,...,troca,elegivel_revisao,dono_aceita_troca,veiculo_único_dono,revisoes_concessionaria,ipva_pago,veiculo_licenciado,garantia_de_fábrica,revisoes_dentro_agenda,veiculo_alienado
0,"1,35E+37",8.0,NISSAN,VERSA,1.6 16V FLEXSTART V-DRIVE MANUAL,2021,2021.0,20258.0,Manual,4,...,False,False,Aceita troca,NaN,NaN,IPVA pago,Licenciado,Garantia de fábrica,NaN,NaN
1,"3,00E+38",18.0,FIAT,STRADA,1.4 MPI WORKING CS 8V FLEX 2P MANUAL,2021,2021.0,53045.0,Manual,2,...,False,False,Aceita troca,Único dono,NaN,IPVA pago,Licenciado,NaN,NaN,NaN
2,"3,16E+38",8.0,AUDI,Q5,2.0 TFSI GASOLINA BLACK S TRONIC,2018,2019.0,32486.0,Automática,4,...,False,False,Aceita troca,NaN,NaN,IPVA pago,NaN,NaN,NaN,NaN
3,"2,23E+38",16.0,CHEVROLET,CRUZE,1.4 TURBO LT 16V FLEX 4P AUTOMÁTICO,2019,2020.0,24937.0,Automática,4,...,False,False,Aceita troca,Único dono,NaN,IPVA pago,Licenciado,NaN,NaN,NaN
4,"1,60E+38",8.0,FORD,ECOSPORT,1.5 TI-VCT FLEX SE AUTOMÁTICO,2019,2019.0,62074.0,Automática,4,...,False,False,Aceita troca,NaN,NaN,IPVA pago,Licenciado,NaN,NaN,NaN


## Pré procesamento na base teste

## Carregando os dados e dropando as colunas que não serão usadas

In [568]:
X = df_test.drop(["id","tipo","marca",'veiculo_alienado', 'elegivel_revisao','tipo_vendedor', 'entrega_delivery', 'revisoes_concessionaria'], axis = 1)

In [569]:
X.shape

(9862, 20)

## Preenchendo valores faltantes

In [570]:
X["num_fotos"].fillna(0, inplace=True)

X["dono_aceita_troca"].fillna(0, inplace=True)
X["dono_aceita_troca"][X["dono_aceita_troca"] == 'Aceita troca'] = 1

X["veiculo_único_dono"].fillna(0, inplace=True)
X["veiculo_único_dono"][X["veiculo_único_dono"] == 'Único dono'] = 1

X["ipva_pago"].fillna(0, inplace=True)
X["ipva_pago"][X["ipva_pago"] == 'IPVA pago'] = 1


X["veiculo_licenciado"].fillna(0, inplace=True)
X["veiculo_licenciado"][X["veiculo_licenciado"] == 'Licenciado'] = 1


X["garantia_de_fábrica"].fillna(0, inplace=True)
X["garantia_de_fábrica"][X["garantia_de_fábrica"] == 'Garantia de fábrica'] = 1

X["revisoes_dentro_agenda"].fillna(0, inplace=True)
X["revisoes_dentro_agenda"][X["revisoes_dentro_agenda"] == 'Todas as revisões feitas pela agenda do carro'] = 1


<ipython-input-570-4efbf8ac31f2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["dono_aceita_troca"][X["dono_aceita_troca"] == 'Aceita troca'] = 1
<ipython-input-570-4efbf8ac31f2>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["veiculo_único_dono"][X["veiculo_único_dono"] == 'Único dono'] = 1
<ipython-input-570-4efbf8ac31f2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["ipva_pago"][X["ipva_pago"] == 'IPVA pago'] = 1
<ipy

In [578]:
X[["num_fotos", "dono_aceita_troca", "veiculo_único_dono","ipva_pago", "veiculo_licenciado", "garantia_de_fábrica", "revisoes_dentro_agenda"]]

,num_fotos,dono_aceita_troca,veiculo_único_dono,ipva_pago,veiculo_licenciado,garantia_de_fábrica,revisoes_dentro_agenda
0,8.0,1,0,1,1,1,0
1,18.0,1,1,1,1,0,0
2,8.0,1,0,1,0,0,0
3,16.0,1,1,1,1,0,0
4,8.0,1,0,1,1,0,0
...,...,...,...,...,...,...,...
9857,8.0,0,0,1,1,0,0
9858,17.0,1,0,0,0,0,0
9859,8.0,0,1,1,0,1,0
9860,9.0,1,1,1,1,0,0


## Ajustando variaveis binárias

In [573]:
X["blindado"][X["blindado"] == 'N'] = 0
X["blindado"][X["blindado"] == 'S'] = 1



X["troca"][X["troca"] == False] = 0
X["troca"][X["troca"] == True] = 1

<ipython-input-573-73eefacdcedc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["blindado"][X["blindado"] == 'N'] = 0
<ipython-input-573-73eefacdcedc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["blindado"][X["blindado"] == 'S'] = 1
<ipython-input-573-73eefacdcedc>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["troca"][X["troca"] == False] = 0


In [577]:
X[["troca", "blindado"]]

,troca,blindado
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
9857,0,0
9858,0,0
9859,0,0
9860,0,0


##Ajustando tipo de coluna

In [579]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9862 entries, 0 to 9861
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   num_fotos               9862 non-null   float64
 1   modelo                  9862 non-null   object 
 2   versao                  9862 non-null   object 
 3   ano_de_fabricacao       9862 non-null   int64  
 4   ano_modelo              9862 non-null   float64
 5   hodometro               9862 non-null   float64
 6   cambio                  9862 non-null   object 
 7   num_portas              9862 non-null   int64  
 8   blindado                9862 non-null   object 
 9   cor                     9862 non-null   object 
 10  cidade_vendedor         9862 non-null   object 
 11  estado_vendedor         9862 non-null   object 
 12  anunciante              9862 non-null   object 
 13  troca                   9862 non-null   object 
 14  dono_aceita_troca       9862 non-null   

In [580]:
X[["troca","dono_aceita_troca","veiculo_único_dono","ipva_pago","veiculo_licenciado",
"garantia_de_fábrica","revisoes_dentro_agenda","blindado"]]= X[["troca","dono_aceita_troca","veiculo_único_dono","ipva_pago","veiculo_licenciado",
"garantia_de_fábrica","revisoes_dentro_agenda","blindado"]].astype(float)

In [581]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9862 entries, 0 to 9861
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   num_fotos               9862 non-null   float64
 1   modelo                  9862 non-null   object 
 2   versao                  9862 non-null   object 
 3   ano_de_fabricacao       9862 non-null   int64  
 4   ano_modelo              9862 non-null   float64
 5   hodometro               9862 non-null   float64
 6   cambio                  9862 non-null   object 
 7   num_portas              9862 non-null   int64  
 8   blindado                9862 non-null   float64
 9   cor                     9862 non-null   object 
 10  cidade_vendedor         9862 non-null   object 
 11  estado_vendedor         9862 non-null   object 
 12  anunciante              9862 non-null   object 
 13  troca                   9862 non-null   float64
 14  dono_aceita_troca       9862 non-null   

In [582]:
X_te = TE_encoder.transform(X[["modelo","versao","cor","cidade_vendedor","estado_vendedor","anunciante","cambio"]])

In [584]:
X_te.head(5)

,modelo,versao,cor,cidade_vendedor,estado_vendedor,anunciante,cambio
0,78198.422077,113698.475007,130277.172282,132822.918819,129709.574763,138703.595663,81204.254603
1,73340.786742,69877.274284,130277.172282,136104.126606,91760.225477,123044.935426,81204.254603
2,179945.214544,146762.751436,130277.172282,164681.673988,153076.079648,138703.595663,145945.090970
3,87613.609976,81893.147445,196878.577634,121932.714183,129709.574763,123044.935426,145945.090970
4,92747.260973,108650.815894,130277.172282,131957.724186,129709.574763,123044.935426,145945.090970


In [585]:
X = X.drop(["modelo","versao","cor","cidade_vendedor","estado_vendedor","anunciante",'cambio'], axis = 1)
X=X.merge(X_te, left_index=True,right_index=True)

X.head()

,num_fotos,ano_de_fabricacao,ano_modelo,hodometro,num_portas,blindado,troca,dono_aceita_troca,veiculo_único_dono,ipva_pago,veiculo_licenciado,garantia_de_fábrica,revisoes_dentro_agenda,modelo,versao,cor,cidade_vendedor,estado_vendedor,anunciante,cambio
0,8.0,2021,2021.0,20258.0,4,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,78198.422077,113698.475007,130277.172282,132822.918819,129709.574763,138703.595663,81204.254603
1,18.0,2021,2021.0,53045.0,2,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,73340.786742,69877.274284,130277.172282,136104.126606,91760.225477,123044.935426,81204.254603
2,8.0,2018,2019.0,32486.0,4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,179945.214544,146762.751436,130277.172282,164681.673988,153076.079648,138703.595663,145945.090970
3,16.0,2019,2020.0,24937.0,4,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,87613.609976,81893.147445,196878.577634,121932.714183,129709.574763,123044.935426,145945.090970
4,8.0,2019,2019.0,62074.0,4,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,92747.260973,108650.815894,130277.172282,131957.724186,129709.574763,123044.935426,145945.090970


## Fazendo a previsão

In [586]:
pred_final=lgbm.predict(X)

In [587]:
pred_final

array([ 82931.00655863,  78587.13647921, 205002.09021831, ...,
        78200.23057043, 129946.96260042, 262168.5630513 ])

In [588]:
subm = pd.DataFrame()
subm['preco'] = pred_final

In [592]:
subm.reset_index(inplace=True)

In [594]:
subm.rename(columns={'index': 'id'}, inplace = True)

In [595]:
subm

,id,preco
0,0,82931.006559
1,1,78587.136479
2,2,205002.090218
3,3,92774.122795
4,4,107719.481734
...,...,...
9857,9857,73615.847120
9858,9858,272698.551324
9859,9859,78200.230570
9860,9860,129946.962600


In [596]:
subm.to_csv('predicted.csv', index = False)